In [1]:
from golden import *
import pandas as pd
import numpy as np
import os

In [2]:
df_poc_class_score = pd.read_csv('/mnt/c/Users/anderson.paiva/Documents/03.3 Squad Saúde/Projeto Neurotech/base_poc_com_class_score.csv')

In [ ]:
df_poc_class_score.info()

In [3]:
# Iniciando o client para acessar o BQ
client_bq = config_client('prod')

# Verificando se o client está conectado
check_client_connection(client_bq)

# Declarando a query que vamos utilizar
query = """
select
  coalesce(pt.user_pet_id, 0) as user_pet_id,
  coalesce(ustp.id, 0) as id_atendimento,
  coalesce(ust.id, 0) as id_procedimentos,
  coalesce(pnf.st_display_name, 'NA') as nome_procedimento,
  coalesce(pt.vl_amount, 0) as vl_amount,
  coalesce(pt.vl_coparticipation, 0) as vl_coparticipation,
  coalesce(pt.clinic_id, 0) as clinic_id,
  coalesce(pt.is_under_analysis, 0) as is_under_analysis,
  coalesce(pt.refuse_procedure_id, 0) as refuse_procedure_id, 
  coalesce(pt.refused_value, 0) as refused_value,
  coalesce(pt.review_status, 0) as review_status,
  coalesce(pt.under_analysis_reason, 'NA') as under_analysis_reason,
  coalesce(rp.reason, 'NA') as reason,
  coalesce(pt.user_observation, 'NA') as user_observation,
  coalesce(pt.dt_created_at, '2000-01-01') as dt_created_at
from `curated_health.health_partner_transfer` pt
left join `petlove-dataeng-prod-01.curated_health.health_user_pet_treatment_procedure` ustp 
  on ustp.id = pt.user_pet_treatment_procedure_id
  and ustp.refdate <= current_date
left join `petlove-dataeng-prod-01.curated_health.health_procedure_nf` pnf
  on pnf.id = ustp.procedure_nf_id
  and pnf.refdate <= current_date
left join `petlove-dataeng-prod-01.curated_health.health_user_pet_treatment` ust
  on ust.id = ustp.user_pet_treatment_id
  and ust.refdate <= current_date
left join `curated_health.health_refuse_procedure` rp 
  on pt.refuse_procedure_id = rp.id
  and rp.refdate <= current_date
where 1=1
  and pt.refdate <= current_date
  and pt.dt_created_at between '2023-11-01' and '2024-03-31'
"""

query_bytes(query, client_bq)

/home/andersonxpm/.local/lib/python3.8/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Quer matar o Cesar????


'1.28 GB'

In [4]:
df_regras_sistemas = run_query(query, client_bq)

Quer matar o Cesar????
Esta consulta processará 1.28 GB.


Deseja executar a consulta? (Y/N) Y


Iniciando query
Iniciando a consulta: 2024-09-07 10:42:37
Consulta concluída: 2024-09-07 10:47:22
Duração da consulta: 0:04:44.934419


In [5]:
df_regras_sistemas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2184508 entries, 0 to 2184507
Data columns (total 15 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   user_pet_id            int64         
 1   id_atendimento         int64         
 2   id_procedimentos       int64         
 3   nome_procedimento      object        
 4   vl_amount              float64       
 5   vl_coparticipation     float64       
 6   clinic_id              int64         
 7   is_under_analysis      int64         
 8   refuse_procedure_id    int64         
 9   refused_value          float64       
 10  review_status          int64         
 11  under_analysis_reason  object        
 12  reason                 object        
 13  user_observation       object        
 14  dt_created_at          datetime64[us]
dtypes: datetime64[us](1), float64(3), int64(7), object(4)
memory usage: 250.0+ MB


In [6]:
df_poc_score_regras = pd.merge(df_poc_class_score, right = df_regras_sistemas, how = 'left', left_on = 'ID_EVENTO', right_on = 'id_atendimento')

In [7]:
df_poc_score_regras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25053 entries, 0 to 25052
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   SCORE                   25053 non-null  float64       
 1   ID_EVENTO               25053 non-null  int64         
 2   ID_ATENDIMENTO          25053 non-null  int64         
 3   DATA_SOLICITACAO        25053 non-null  object        
 4   STATUS                  25053 non-null  object        
 5   VL_REPASSE              25053 non-null  float64       
 6   MOTIVO_GLOSA            3803 non-null   object        
 7   FL_EDICAO_PROCEDIMENTO  25053 non-null  int64         
 8   DIFF_APOS_EDICAO        25040 non-null  float64       
 9   STATUS_GLOSA            3801 non-null   object        
 10  CLASSIFICACAO_SCORE     25053 non-null  object        
 11  user_pet_id             25053 non-null  int64         
 12  id_atendimento          25053 non-null  int64 

In [8]:
save_to_csv(df_poc_score_regras, path = '/mnt/c/Users/anderson.paiva/Downloads', file_name = 'poc_score_regras_20240907')

'Arquivo poc_score_regras_20240907.csv salvo com sucesso em /mnt/c/Users/anderson.paiva/Downloads/poc_score_regras_20240907.csv!!'

In [ ]:
df_poc_score_regras_vf = df_poc_score_regras[['SCORE',
                                              'ID_EVENTO',
                                              'ID_ATENDIMENTO',
                                              'DATA_SOLICITACAO',
                                              'VL_REPASSE',
                                              'MOTIVO_GLOSA',
                                              'FL_EDICAO_PROCEDIMENTO',
                                              'status_novo_y',
                                              'DIFF_APOS_EDICAO',
                                              '',
                                              '',
                                              '',
                                              '',
                                              '',
                                              '',
                                              '',
                                              '',
                                              '',
                                             ]]

In [ ]:
df_final

In [ ]:
# Agora vamos criar a coluna com as faixas de score (Categorização do score)
condicoes = [
    (df_uniao['SCORE']<= .1),
    (df_uniao['SCORE']>.1) & (df_uniao['SCORE']<=.2),
    (df_uniao['SCORE']>.2) & (df_uniao['SCORE']<=.3),
    (df_uniao['SCORE']>.3) & (df_uniao['SCORE']<=.4),
    (df_uniao['SCORE']>.4) & (df_uniao['SCORE']<=.5),
    (df_uniao['SCORE']>.5) & (df_uniao['SCORE']<=.6),
    (df_uniao['SCORE']>.6) & (df_uniao['SCORE']<=.7),
    (df_uniao['SCORE']>.7) & (df_uniao['SCORE']<=.8),
    (df_uniao['SCORE']>.8) & (df_uniao['SCORE']<=.9),
    (df_uniao['SCORE']>.9)
]

categorias = [
    '0 - 0 a 10%',
    '1 - 10 a 20%',
    '2 - 20 a 30%',
    '3 - 30 a 40%',
    '4 - 40 a 50%',
    '5 - 50 a 60%',
    '6 - 60 a 70%',
    '7 - 70 a 80%',
    '8 - 80 a 90%',
    '9 - 90 a 100%',
]
df_uniao['CLASSIFICACAO_SCORE'] = np.select(condicoes, categorias, np.nan)
df_uniao.info()

In [ ]:
df_uniao = df_uniao.drop(['id_atendimento', 'numero_atendimento', 'data_origem_date', 'nome_procedimento'], axis = 1)

In [ ]:
df_uniao = df_uniao.rename(columns = {'PREV ANALISADO': 'FL_JA_ANALISADO', 'status_novo': 'STATUS', 'vl_bruto': 'VL_REPASSE', 'motivo_glosa_desc': 'MOTIVO_GLOSA', 'fl_edicao_procedimento':'FL_EDICAO_PROCEDIMENTO', 'diff_apos_edicao': 'DIFF_APOS_EDICAO', 'status_glosa':'STATUS_GLOSA'})

In [ ]:
df_uniao.info()

In [ ]:
save_to_csv(df_uniao, path = '/mnt/c/Users/anderson.paiva/Downloads', file_name = 'base_poc_com_class_score')

------------------------